# Context Encoder 

Let's begin by importing tensorflow and the network

In [ ]:
import tensorflow as tf
from network.contextEncoder import ContextEncoderNetwork

Now we initialize the context encoder network and select the step we want to use for the reconstruction.

In [ ]:
tf.reset_default_graph()

train_filename = 'train_full_w5120_g1024_h512_19404621.tfrecords'
valid_filename = 'valid_full_w5120_g1024_h512_ex913967.tfrecords'

aContextEncoderNetwork = ContextEncoderNetwork(batch_size=256, window_size=5120, gap_length=1024, 
                                             learning_rate=1e-5, name='test')

In [ ]:
best_step = 506000 

In [ ]:
test_filename = 'test_full_w5120_g1024_h512_ex298385.tfrecords'
reconstructed, out_gaps = anAutoEncoderNetwork.reconstruct(test_filename, best_step, max_steps=248)

evaluator = EvaluationWriter(anAutoEncoderNetwork._name + str(best_step) + '_test.xlsx')
evaluator.evaluate(reconstructed, out_gaps, best_step)
evaluator.save()

In [ ]:
from matplotlib.backends.backend_pdf import PdfPages
pp = PdfPages(anAutoEncoderNetwork._name + '_' + str(best_step) + '_test.pdf')

pylab.rcParams['figure.figsize'] = (14, 28)
f, axarr = plt.subplots(8, 2, sharey='row')

stop_value = min(256, len(out_gaps)+1)
for i in range(0, stop_value):   
    if i is not 0 and i%8 is 0:
        pp.savefig()
        f, axarr = plt.subplots(8, 2, sharey='row')
    axarr[i%8, 0].plot(out_gaps[i%256])
    axarr[i%8, 1].plot(reconstructed[i%256]) 
    
pp.savefig()
pp.close()

In [ ]:
def _pavlovs_SNR(y_orig, y_inp):
    norm_y_orig = np.linalg.norm(y_orig) + 1e-10
    norm_y_orig_minus_y_inp = np.linalg.norm(y_orig - y_inp)
    return 10 * np.log10((abs(norm_y_orig ** 2)) / abs((norm_y_orig_minus_y_inp ** 2)))

def _euclideanNorm(vector):
        squared = np.square(vector)
        summed = np.sum(squared, axis=1)
        return np.sqrt(summed + 1e-10)
    
fake_a = (reconstructed - 0.5) * 2
gap = (out_gaps - 0.5) * 2

SNRs = np.zeros((len(fake_a),))
for index, signal in enumerate(fake_a):
    SNRs[index] = _pavlovs_SNR(gap[index], fake_a[index])

norm_orig = _euclideanNorm(gap)
error = (gap - fake_a)
reconstruction_loss = 0.5 * np.sum(np.square(error), axis=1) * (1 + 1 / norm_orig)


In [ ]:
pylab.rcParams['figure.figsize'] = (14, 8)

import scipy.stats as stats
sorted_SNR = sorted(SNRs)

fit = stats.norm.pdf(sorted_SNR, np.mean(sorted_SNR), np.std(sorted_SNR))  #this is a fitting indeed

plt.plot(sorted_SNR,fit,'-o')

plt.hist(sorted_SNR, 50, normed=True)    


In [ ]:
plt.scatter(SNRs, reconstruction_loss)

In [ ]:
pylab.rcParams['figure.figsize'] = (14, 28)

examples = np.where((SNRs<-10) & (reconstruction_loss<5))[0]

f, axarr = plt.subplots(8, 2, sharey='row')

for index, example in enumerate(examples):   
    if index is not 0 and index%8 is 0:
        f, axarr = plt.subplots(8, 2, sharey='row')
    axarr[index%8, 0].plot(out_gaps[example])
    axarr[index%8, 1].plot(reconstructed[example]) 


In [ ]:
examples = np.where((SNRs<1) & (SNRs>-1))[0][:256]

f, axarr = plt.subplots(8, 2, sharey='row')

for index, example in enumerate(examples):   
    if index is not 0 and index%8 is 0:
        f, axarr = plt.subplots(8, 2, sharey='row')
    axarr[index%8, 0].plot(out_gaps[example])
    axarr[index%8, 1].plot(reconstructed[example]) 
    

In [ ]:
examples = np.where((SNRs>28))[0][:256]

f, axarr = plt.subplots(8, 2, sharey='row')

for index, example in enumerate(examples):   
    if index%8 == 0:
        f, axarr = plt.subplots(8, 2, sharey='row')
    axarr[index%8, 0].plot(out_gaps[example])
    axarr[index%8, 1].plot(reconstructed[example]) 
    